In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.style
matplotlib.style.use("seaborn")
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import seaborn as sns
sns.color_palette("hls", 17)
import scipy.stats as st
import math

from pingouin import rcorr
import pingouin as pg

from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.multioutput import RegressorChain, MultiOutputRegressor
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn import linear_model
from sklearn import svm 
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_validate, cross_val_score, learning_curve
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score, plot_roc_curve, roc_curve, auc, mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn import feature_selection #import chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
import sys
import os

from sys import platform
if platform == "darwin":
    sys.path.append(os.path.dirname(os.path.realpath(__file__)) + "/..")
    smart_nlp_path = ''
elif platform == "win32":
    sys.path.append('../')
    smart_nlp_path = os.getcwd()
    smart_nlp_path = "\\".join([smart_nlp_path.split("\\")[i] for i in range(0,len(smart_nlp_path.split("\\"))-1)]+["/"])

from module.trend_analysis_functions import *
from module.topic_model_plus_class import Topic_Model_plus

In [3]:
data = pd.read_csv(smart_nlp_path+"input data/Cleaned_ICS_predictive_regression_sitreps.csv").drop(["Unnamed: 0.1","Unnamed: 0"], axis=1)

C:\Users\srandrad\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3050: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
data

,CY,DISCOVERY_DATE,INCIDENT_ID,PCT_CONTAINED_COMPLETED,START_YEAR,TOTAL_AERIAL,TOTAL_PERSONNEL,REPORT_DOY,DISCOVERY_DOY,Combined_Text,...,Current_total_Structures_Destroyed,Current_total_Fatalities,Diff_Injuries,Diff_Structures_Damages,Diff_Structures_Destroyed,Diff_Fatalities,Total_Injuries,Total_Structures_Damages,Total_Structures_Destroyed,Total_Fatalities
0,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,80.0,2010.0,5.000000,230.000000,197,196,"['resource', 'share', 'cactus']",...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,60.0,2010.0,5.000000,230.000000,197,196,"['resource', 'share', 'incident', 'cactus', 'i...",...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,30.0,2010.0,4.000000,165.000000,197,196,"['resource', 'share', 'cactus', 'erratic', 'wi...",...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,100.0,2010.0,4.333333,192.333333,197,196,"['resource', 'share', 'cactus', 'cactus', 'bec...",...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,60.0,2010.0,4.333333,192.333333,197,196,"['resource', 'share', 'cactus', 'cactus', 'bec...",...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44338,2014,2014-03-15 14:30:00,2014_VAVAS1403037_BEAVER LODGE RD.,100.0,2014.0,0.000000,13.000000,74,74,"['fast', 'spread', 'field']",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
44339,2014,2014-03-19 14:00:00,2014_VAVAS1406037_AIRPORT MOUNTAIN,85.0,2014.0,0.000000,18.500000,80,78,"['heavy', 'plume', 'primary', 'carrier']",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44340,2014,2014-08-20 13:00:00,2014_WA-WFS-513_SAND RIDGE,0.0,2014.0,1.000000,95.000000,234,232,"['heavy', 'canyon', 'river', 'mainly', 'canyon...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44341,2014,2014-08-20 13:00:00,2014_WA-WFS-513_SAND RIDGE,86.0,2014.0,1.000000,120.000000,235,232,"['laid', 'night', 'test', 'wind', 'remain', 'c...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data.columns

Index(['CY', 'DISCOVERY_DATE', 'INCIDENT_ID', 'PCT_CONTAINED_COMPLETED',
       'START_YEAR', 'TOTAL_AERIAL', 'TOTAL_PERSONNEL', 'REPORT_DOY',
       'DISCOVERY_DOY', 'Combined_Text', 'Unique_IDs', 'ACRES', 'WF_FSR',
       'INJURIES', 'FATALITIES', 'EST_IM_COST_TO_DATE', 'STR_DAMAGED',
       'STR_DESTROYED', 'NEW_ACRES', 'POO_STATE', 'POO_LATITUDE',
       'POO_LONGITUDE', 'WEATHER_CONCERNS_NARR', 'INC_MGMT_ORG_ABBREV',
       'EVACUATION_IN_PROGRESS', 'Incident_region', 'Traffic',
       'Command_Transitions', 'Evacuations', 'Inaccurate_Mapping',
       'Aerial_Grounding', 'Resource_Issues', 'Injuries', 'Cultural_Resources',
       'Livestock', 'Law_Violations', 'Military_Base', 'Infrastructure',
       'Extreme_Weather', 'Ecological', 'Hazardous_Terrain', 'Floods',
       'Dry_Weather', 'Severity', 'NUM_REPORTS', 'DAYS_BURING',
       'Total_Incident_Text', 'Current_total_Injuries',
       'Current_total_Structures_Damages',
       'Current_total_Structures_Destroyed', 'Current_tot

In [6]:
X = data[['TOTAL_AERIAL', 'TOTAL_PERSONNEL', 'WF_FSR', 'DAYS_BURING', 'ACRES','PCT_CONTAINED_COMPLETED', 'Current_total_Injuries',
       'Current_total_Structures_Damages',
       'Current_total_Structures_Destroyed', 'Current_total_Fatalities','Traffic',
       'Command_Transitions', 'Evacuations', 'Inaccurate_Mapping',
       'Aerial_Grounding', 'Resource_Issues', 'Injuries', 'Cultural_Resources',
       'Livestock', 'Law_Violations', 'Military_Base', 'Infrastructure',
       'Extreme_Weather', 'Ecological', 'Hazardous_Terrain', 'Floods',
       'Dry_Weather']]
y = data[['Diff_Injuries', 'Diff_Structures_Damages', 'Diff_Structures_Destroyed',
       'Diff_Fatalities']]
y

,Diff_Injuries,Diff_Structures_Damages,Diff_Structures_Destroyed,Diff_Fatalities
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
44338,0.0,0.0,0.0,0.0
44339,0.0,0.0,0.0,0.0
44340,0.0,0.0,0.0,0.0
44341,0.0,0.0,0.0,0.0


In [7]:
Xtrain_val, Xtest, ytrain_val, ytest = train_test_split(X,y, random_state=42, test_size=0.1, shuffle=True)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain_val, ytrain_val, random_state=42, test_size=(0.1/0.9), shuffle=True)

In [8]:
#minmax_scale X
minMaxScaler_cls=MinMaxScaler()
minMaxScaler_cls.fit(Xtrain)
norm_Xtrain=minMaxScaler_cls.transform(Xtrain)
norm_Xvalid=minMaxScaler_cls.transform(Xval)
norm_Xtest=minMaxScaler_cls.transform(Xtest)

Models to test:
- SVM
- NB
- Logistic Regression
- Random Forest
- xgboost
- AdaBoost
- NN

In [9]:
#svm
mean_squared_error(ytrain,MultiOutputRegressor(svm.LinearSVR()).fit(norm_Xtrain, ytrain).predict(norm_Xtrain))

329302.7365561884

In [10]:
MultiOutputRegressor(svm.LinearSVR()).fit(norm_Xtrain, ytrain).score(norm_Xtrain, ytrain)

0.1585569945667253

In [11]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
model_names = ["SVM", "GradientBoosting", "Random Forest", "Bayesian Ridge"]
train_score = []; test_score = []; train_MSE = []; test_MSE = []
models = [svm.LinearSVR(), GradientBoostingRegressor(random_state=0), RandomForestRegressor(max_depth=3, random_state=0),
         linear_model.BayesianRidge()]
for m in models:
    mdl = MultiOutputRegressor(m)
    mdl.fit(norm_Xtrain, ytrain)
    train_score.append(round(mdl.score(norm_Xtrain, ytrain),3))
    test_score.append(round(mdl.score(norm_Xtest, ytest),3))
    train_MSE.append(round(mean_squared_error(ytrain, mdl.predict(norm_Xtrain)),3))
    test_MSE.append(round(mean_squared_error(ytest, mdl.predict(norm_Xtest)),3))

results_df = pd.DataFrame({"model": model_names,
                          "train score": train_score,
                          "test score": test_score,
                          "train MSE": train_MSE,
                          "test MSE": test_MSE})
results_df

,model,train score,test score,train MSE,test MSE
0,SVM,0.159,0.159,329298.899,316373.464
1,GradientBoosting,1.000,0.999,121.855,411.597
2,Random Forest,0.994,0.993,2498.478,3102.901
3,Bayesian Ridge,0.999,0.999,497.312,687.734
